In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

# 🔧 제외할 상세정보 항목
EXCLUDE_KEYS = {"유통회사", "등록년월", "안전확인인증", "적합성평가인증"}

# -----------------------------------------------------------------------------
# 브라우저 초기화
# -----------------------------------------------------------------------------
def init_driver(headless=True):
    options = webdriver.ChromeOptions()
    if headless:
        options.add_argument("--headless=new")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--lang=ko-KR")

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    driver.set_window_size(1400, 1000)
    return driver

# -----------------------------------------------------------------------------
# 1️⃣ 기본 리스트 페이지에서 제품명/가격/pcode 추출
# -----------------------------------------------------------------------------
def get_basic_info_from_page(soup):
    products = []
    items = soup.select(".prod_item")

    for item in items:
        try:
            title_elem = item.select_one(".prod_name a")
            title = title_elem.get_text(strip=True) if title_elem else "N/A"

            # 가격
            price_elem = item.select_one(".price_sect strong") or item.select_one(".price_sect em")
            price = price_elem.get_text(strip=True) if price_elem else "N/A"

            # 제품 코드(pcode)
            pcode = None
            if title_elem and "href" in title_elem.attrs:
                href = title_elem["href"]
                if "pcode=" in href:
                    pcode = href.split("pcode=")[1].split("&")[0]

            if title != "N/A" and pcode:
                products.append({"제품명": title, "가격": price, "pcode": pcode})

        except Exception as e:
            print(f"⚠️ 항목 스킵: {e}")
            continue

    return products

# -----------------------------------------------------------------------------
# 2️⃣ 상세페이지에서 스펙 추출 (특정 항목 제외)
# -----------------------------------------------------------------------------
def get_detail_info(driver, pcode):
    try:
        detail_url = f'https://prod.danawa.com/info/?pcode={pcode}'
        driver.get(detail_url)
        time.sleep(3)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        specs = {}
        manufacturer = 'N/A'

        # ✅ 모든 spec 영역 탐색 (spec_tbl, prod_spec 둘 다 대응)
        tables = soup.select('.spec_tbl, .prod_spec')

        for table in tables:
            rows = table.select('tr')
            for row in rows:
                th_elems = row.select('th')
                td_elems = row.select('td')

                # ✅ th만 있고 td가 없는 경우도 있음 → 처리
                if th_elems and not td_elems:
                    for th in th_elems:
                        key = th.get_text(strip=True)
                        specs[key] = '정보없음'
                    continue

                # ✅ 일반적인 th–td 매칭
                for th, td in zip(th_elems, td_elems):
                    key = th.get_text(strip=True)
                    value = td.get_text(" ", strip=True)
                    specs[key] = value

        # ✅ 제외할 항목 필터링
        exclude_keys = {'등록년월', '유통회사', '안전확인인증', '적합성평가인증'}
        specs = {k: v for k, v in specs.items() if k not in exclude_keys}

        # ✅ 제조회사 별도 추출
        if '제조회사' in specs:
            manufacturer = specs.pop('제조회사')

        return manufacturer, specs

    except Exception as e:
        print(f"    ⚠️ 상세 페이지 에러: {e}")
        return 'N/A', {}


# -----------------------------------------------------------------------------
# 3️⃣ 페이지 이동 (1~10 → 다음 → 11~20 ...)
# -----------------------------------------------------------------------------
def click_next_page(driver, current_page):
    try:
        next_page = current_page + 1
        xpath = f'//a[@onclick="javascript:movePage({next_page}); return false;"]'
        next_btn = driver.find_element(By.XPATH, xpath)
        driver.execute_script("arguments[0].click();", next_btn)
        time.sleep(3)
        return True

    except NoSuchElementException:
        try:
            # 10페이지 단위로 다음 그룹 이동
            next_block = driver.find_element(By.XPATH, '//a[@class="btn_num btn_next"]')
            driver.execute_script("arguments[0].click();", next_block)
            time.sleep(3)
            return True
        except NoSuchElementException:
            print(f"✅ 마지막 페이지 도달")
            return False
    except Exception as e:
        print(f"⚠️ 페이지 이동 실패: {e}")
        return False

# -----------------------------------------------------------------------------
# 4️⃣ 모든 페이지 순회하며 기본 정보 수집
# -----------------------------------------------------------------------------
def crawl_all_pages(driver, base_url, max_products=99999):
    all_products = []
    page = 1

    url = base_url + "&sort=12"
    driver.get(url)
    time.sleep(4)

    while len(all_products) < max_products:
        print(f"\n📄 페이지 {page} 크롤링 중...")

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

        soup = BeautifulSoup(driver.page_source, "html.parser")
        page_products = get_basic_info_from_page(soup)

        if not page_products:
            print("✅ 더 이상 상품 없음")
            break

        print(f"✅ {len(page_products)}개 상품 발견")
        all_products.extend(page_products)

        if not click_next_page(driver, page):
            break

        page += 1

    return all_products

# -----------------------------------------------------------------------------
# 5️⃣ 상세 정보 병합
# -----------------------------------------------------------------------------
def add_detail_info(driver, products):
    print(f"\n{'='*70}")
    print("STEP 2: 상세 정보 크롤링")
    print(f"{'='*70}\n")
    
    total = len(products)
    
    for i, product in enumerate(products, 1):
        try:
            print(f"\n[{i}/{total}] {product['제품명'][:50]}...")

            if product['pcode']:
                manufacturer, specs = get_detail_info(driver, product['pcode'])
                product['제조사'] = manufacturer
                product['상세정보'] = str(specs)

                if specs:
                    preview = list(specs.items())[:2]
                    for k, v in preview:
                        print(f"   - {k}: {v[:40]}...")

                time.sleep(2)
            else:
                product['제조사'] = 'N/A'
                product['상세정보'] = '{}'

        except Exception as e:
            print(f"    ❌ 에러: {e}")
            product['제조사'] = 'N/A'
            product['상세정보'] = '{}'

        # ✅ 10개 단위로 중간 저장
        if i % 10 == 0:
            temp_df = pd.DataFrame(products[:i])
            temp_df = temp_df[['제품명', '가격', '제조사', '상세정보']]
            temp_df.to_csv('중간저장.csv', index=False, encoding='utf-8-sig')
            print(f"\n  📁 중간 저장 완료: {i}개 수집됨 (cpu_partial.csv)")

        if i % 10 == 0:
            print(f"\n  📊 진행률: {i}/{total} ({i/total*100:.1f}%)")

    return products

# -----------------------------------------------------------------------------
# 6️⃣ 실행 메인
# -----------------------------------------------------------------------------
def main():
    print("=" * 70)
    print(" 크롤링 시작")
    print("=" * 70)

    driver = init_driver(headless=False)  # 디버깅 시 False 추천

    try:
        base_url = "https://prod.danawa.com/list/?cate=112777"

        # 1단계: 목록 크롤링
        products = crawl_all_pages(driver, base_url, max_products=99999)
        print(f"\n✅ 기본 정보 {len(products)}개 수집 완료")

        # 2단계: 상세정보 추가
        products = add_detail_info(driver, products)

        # 3단계: 저장
        for p in products:
            p.pop("pcode", None)

        df = pd.DataFrame(products)
        df["상세정보"] = df["상세정보"].apply(lambda x: str(x))

        filename = "power_data_all.csv"
        df.to_csv(filename, index=False, encoding="utf-8-sig")

        print(f"\n✅ CSV 저장 완료: {filename}")
        print(f"총 {len(df)}개 상품")

    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        import traceback
        traceback.print_exc()
    finally:
        driver.quit()
        print("\n크롤링 종료 ✅")

# -----------------------------------------------------------------------------
if __name__ == "__main__":
    main()

 크롤링 시작

📄 페이지 1 크롤링 중...
✅ 36개 상품 발견

📄 페이지 2 크롤링 중...
✅ 30개 상품 발견

📄 페이지 3 크롤링 중...
✅ 30개 상품 발견

📄 페이지 4 크롤링 중...
✅ 30개 상품 발견

📄 페이지 5 크롤링 중...
✅ 30개 상품 발견

📄 페이지 6 크롤링 중...
✅ 30개 상품 발견

📄 페이지 7 크롤링 중...
✅ 30개 상품 발견

📄 페이지 8 크롤링 중...
✅ 30개 상품 발견

📄 페이지 9 크롤링 중...
✅ 30개 상품 발견

📄 페이지 10 크롤링 중...
✅ 30개 상품 발견

📄 페이지 11 크롤링 중...
✅ 30개 상품 발견

📄 페이지 12 크롤링 중...
✅ 30개 상품 발견

📄 페이지 13 크롤링 중...
✅ 30개 상품 발견

📄 페이지 14 크롤링 중...
✅ 30개 상품 발견

📄 페이지 15 크롤링 중...
✅ 30개 상품 발견

📄 페이지 16 크롤링 중...
✅ 30개 상품 발견

📄 페이지 17 크롤링 중...
✅ 30개 상품 발견

📄 페이지 18 크롤링 중...
✅ 30개 상품 발견

📄 페이지 19 크롤링 중...
✅ 30개 상품 발견

📄 페이지 20 크롤링 중...
✅ 30개 상품 발견

📄 페이지 21 크롤링 중...
✅ 30개 상품 발견

📄 페이지 22 크롤링 중...
✅ 30개 상품 발견

📄 페이지 23 크롤링 중...
✅ 30개 상품 발견

📄 페이지 24 크롤링 중...
✅ 30개 상품 발견

📄 페이지 25 크롤링 중...
✅ 30개 상품 발견

📄 페이지 26 크롤링 중...
✅ 30개 상품 발견

📄 페이지 27 크롤링 중...
✅ 30개 상품 발견

📄 페이지 28 크롤링 중...
✅ 30개 상품 발견

📄 페이지 29 크롤링 중...
✅ 30개 상품 발견

📄 페이지 30 크롤링 중...
✅ 30개 상품 발견

📄 페이지 31 크롤링 중...
✅ 30개 상품 발견

📄 페이지 32 크롤링 중...
✅ 30개 상품 발견

📄 페이지 33

In [4]:
import pandas as pd 
df = pd.read_csv('cpu.csv')

df

,제품명,가격,제조사,상세정보,텍스트
0,AMD 라이젠7-5세대 7800X3D (라파엘) (멀티팩 정품),"453,640",AMD(제조사 웹사이트 바로가기),"{'AMD CPU종류': '라이젠7-5세대', '코어 수': '8코어', '메모리 ...","AMD 라이젠7-5세대 7800X3D (라파엘) (멀티팩 정품) 453,640 AM..."
1,인텔 코어 울트라5 시리즈2 245K (애로우레이크) (정품),"320,910",인텔(제조사 웹사이트 바로가기),"{'인텔 CPU종류': '코어 울트라5(S2)', '코어 수': 'P6+E8코어',...","인텔 코어 울트라5 시리즈2 245K (애로우레이크) (정품) 320,910 인텔(..."
2,인텔 코어 울트라7 시리즈2 265K (애로우레이크) (정품),"449,530",인텔(제조사 웹사이트 바로가기),"{'인텔 CPU종류': '코어 울트라7(S2)', '코어 수': 'P8+E12코어'...","인텔 코어 울트라7 시리즈2 265K (애로우레이크) (정품) 449,530 인텔(..."
3,AMD 라이젠5-6세대 9600 (그래니트 릿지) (멀티팩 정품),"275,930",AMD(제조사 웹사이트 바로가기),"{'AMD CPU종류': '라이젠5-6세대', '코어 수': '6코어', '메모리 ...","AMD 라이젠5-6세대 9600 (그래니트 릿지) (멀티팩 정품) 275,930 A..."
4,AMD 라이젠5-5세대 7500F (라파엘),"185,470",AMD(제조사 웹사이트 바로가기),"{'AMD CPU종류': '라이젠5-5세대', '코어 수': '6코어', '메모리 ...","AMD 라이젠5-5세대 7500F (라파엘) 185,470 AMD(제조사 웹사이트 ..."
...,...,...,...,...,...
547,인텔 펜티엄 E5300 (울프데일),940,인텔 펜티엄,"{'인텔 CPU종류': '인텔(펜티엄)', '코어 수': '듀얼 코어', '내장그래...",인텔 펜티엄 E5300 (울프데일) 940 인텔 펜티엄 인텔 CPU종류 인텔(펜티엄...
548,인텔 코어2듀오 E8500 (울프데일),"4,700",인텔 코어2듀오,"{'인텔 CPU종류': '인텔(코어2듀오)', '코어 수': '듀얼 코어', '내장...","인텔 코어2듀오 E8500 (울프데일) 4,700 인텔 코어2듀오 인텔 CPU종류 ..."
549,인텔 셀러론 G1630 (아이비브릿지),"2,200",인텔 셀러론,"{'인텔 CPU종류': '인텔(셀러론)', '코어 수': '듀얼 코어', '메모리 ...","인텔 셀러론 G1630 (아이비브릿지) 2,200 인텔 셀러론 인텔 CPU종류 인텔..."
550,인텔 펜티엄 E2200 (콘로),"5,400",인텔 펜티엄,"{'인텔 CPU종류': '인텔(펜티엄)', '코어 수': '듀얼 코어', '내장그래...","인텔 펜티엄 E2200 (콘로) 5,400 인텔 펜티엄 인텔 CPU종류 인텔(펜티엄..."


In [12]:
import pandas as pd
import ast

df = pd.read_csv('sdd_data_all.csv', encoding='utf-8-sig')

def create_embedding_text(row):
    specs = ast.literal_eval(row['상세정보'])
    spec_text = " ".join([f"{k} {v}" for k, v in specs.items()])
    return f"{row['제품명']} {row['가격']} {row['제조사']} {spec_text}"

df['텍스트'] = df.apply(create_embedding_text, axis=1)
df.to_csv('sdd.csv', index=False, encoding='utf-8-sig')

In [11]:
import pandas as pd 

df = pd.read_csv(r'C:\Users\PCN\Desktop\pcn\가공데이터\ssd.csv')
df = df.drop('상세정보',axis=1)
df.to_csv(r'C:\Users\PCN\Desktop\pcn\가공데이터\ssd.csv')

In [19]:
df = df.drop('상세정보',axis=1)
df


,제품명,가격,제조사,텍스트
0,AMD 라이젠7-5세대 7800X3D (라파엘) (멀티팩 정품),"453,640",AMD(제조사 웹사이트 바로가기),"AMD 라이젠7-5세대 7800X3D (라파엘) (멀티팩 정품) 453,640 AM..."
1,인텔 코어 울트라5 시리즈2 245K (애로우레이크) (정품),"320,910",인텔(제조사 웹사이트 바로가기),"인텔 코어 울트라5 시리즈2 245K (애로우레이크) (정품) 320,910 인텔(..."
2,인텔 코어 울트라7 시리즈2 265K (애로우레이크) (정품),"449,530",인텔(제조사 웹사이트 바로가기),"인텔 코어 울트라7 시리즈2 265K (애로우레이크) (정품) 449,530 인텔(..."
3,AMD 라이젠5-6세대 9600 (그래니트 릿지) (멀티팩 정품),"275,930",AMD(제조사 웹사이트 바로가기),"AMD 라이젠5-6세대 9600 (그래니트 릿지) (멀티팩 정품) 275,930 A..."
4,AMD 라이젠5-5세대 7500F (라파엘),"185,470",AMD(제조사 웹사이트 바로가기),"AMD 라이젠5-5세대 7500F (라파엘) 185,470 AMD(제조사 웹사이트 ..."
...,...,...,...,...
547,인텔 펜티엄 E5300 (울프데일),940,인텔 펜티엄,인텔 펜티엄 E5300 (울프데일) 940 인텔 펜티엄 인텔 CPU종류 인텔(펜티엄...
548,인텔 코어2듀오 E8500 (울프데일),"4,700",인텔 코어2듀오,"인텔 코어2듀오 E8500 (울프데일) 4,700 인텔 코어2듀오 인텔 CPU종류 ..."
549,인텔 셀러론 G1630 (아이비브릿지),"2,200",인텔 셀러론,"인텔 셀러론 G1630 (아이비브릿지) 2,200 인텔 셀러론 인텔 CPU종류 인텔..."
550,인텔 펜티엄 E2200 (콘로),"5,400",인텔 펜티엄,"인텔 펜티엄 E2200 (콘로) 5,400 인텔 펜티엄 인텔 CPU종류 인텔(펜티엄..."
